**Load data**

Load from csv into pandas. Each sequence, one by one.

In [1]:
from util.data import create_full_datasets

sequence_length = 60
batch_size = 32
train, validation, test = create_full_datasets('data', sequence_length=sequence_length,
                                                   return_sequence=True, shift=1, batch_size=batch_size)

print('train', len(train))
print('validation', len(validation))
print('test', len(test))

train 500
validation 160
test 1240


**Strategies**
The ML models are evaluated using the following strategies:
* Average return
* Sharpe ratio

**Model**

Initialize Model

In [2]:
from model.lstm import compile_lstm_model
from model.training import sharpe_loss

loss = sharpe_loss
target = 0.15
alpha = 0.1
dropout = 0.5
learning_rate = 1e-4
model = compile_lstm_model(loss=loss, target=target, alpha=alpha, dropout=dropout, learning_rate=learning_rate)

**Training**

Run training.

In [3]:
from model.training import fit

patience = 10
epochs = 100
history = fit(model, train, validation, patience=patience, epochs=epochs)

Epoch 1/100
500/500 - 12s - loss: 2.9647 - val_loss: -1.5701e+00
Epoch 2/100
500/500 - 11s - loss: -4.1151e+00 - val_loss: -3.9379e+00
Epoch 3/100
500/500 - 12s - loss: -1.0849e+02 - val_loss: -2.6554e+00
Epoch 4/100
500/500 - 12s - loss: 1.8463 - val_loss: -2.6528e+00
Epoch 5/100
500/500 - 12s - loss: 17.2200 - val_loss: -2.6571e+00
Epoch 6/100


KeyboardInterrupt: 